In [1]:
pip install torch torchaudio asteroid_filterbanks numpy matplotlib torchlibrosa librosa resampy


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import resampy

import torch
import torchaudio
import asteroid_filterbanks as fb
import numpy as np
import scipy.io.wavfile as wav
from torchlibrosa import STFT
import librosa
import numpy as np
from asteroid_filterbanks import make_enc_dec
from asteroid_filterbanks import transforms 

In [3]:
audio_path = 'dataset_wav\mic_sigs0001.wav'
sr, audio = wav.read(audio_path)

print(f"audio shape: {audio.shape}")
audio = audio.squeeze()
print(f"audio shape: {audio.shape}")
audio = torch.tensor(audio).float()

print(f"audio shape: {audio.shape}")
print(f"audio dim: {audio.ndimension()}")

audio shape: (160000,)
audio shape: (160000,)
audio shape: torch.Size([160000])
audio dim: 1


<>:1: SyntaxWarning: invalid escape sequence '\m'
<>:1: SyntaxWarning: invalid escape sequence '\m'
C:\Users\Comlab\AppData\Local\Temp\ipykernel_2160\2333029042.py:1: SyntaxWarning: invalid escape sequence '\m'
  audio_path = 'dataset_wav\mic_sigs0001.wav'


In [ ]:
def process_audio_with_librosa_stft(audio_path):
    # Carica l'audio
    audio, _ = librosa.load(audio_path, sr=16000, mono = True, res_type = "kaiser_fast")
    
    print(f"audio shape prima di unsqueeze: {audio.shape}")
    stft = STFT(n_fft=512, hop_length=256)

    #audio = audio.squeeze()
    print(f"audio shape prima di unsqueeze: {audio.shape}")


    audio = torch.tensor(audio).float()

    print(f"audio shape post unsqueeze: {audio.shape}")


    # Calcola STFT usando librosa
    
    x_real, x_imm = stft(audio)


    magn = torch.sqrt(torch.pow(x_real, 2) + torch.pow(x_imm, 2))
    magn = torch.log(magn**2 + 1e-7)

    angles_cos = torch.cos(torch.angle(x_real + 1j*x_imm))
    angles_sin = torch.sin(torch.angle(x_real + 1j*x_imm))


    print(f"\n")
    print(f"dimensioni magnitude prima del resizing: {magn.shape}")
    print(f"dimensioni angoli fase prima del resizing: {angles_cos.shape}, {angles_sin.shape}")

    magn = magn[:,:,:,:-1]
    angles_cos = angles_cos[:,:,:,:-1]
    angles_sin = angles_sin[:,:,:,:-1]
    # torch Librosa [1,1,514,624] (batch, canale, frequenze, tempo)

    print(f"\n")

    print(f"dimensioni magnitude dopo del resizing: {magn.shape}")
    print(f"dimensioni angoli fase dopo del resizing: {angles_cos.shape}, {angles_sin.shape}")

process_audio_with_librosa_stft(audio_path)

In [94]:
def process_audio_with_stft_encoder(audio_path):
    # Carica l'audio
    audio, _ = librosa.load(audio_path, sr=16000, mono = True, res_type = "kaiser_fast")
    audio = audio.squeeze()
    audio = torch.tensor(audio).float()

    #print(f"forma audio encoded con parte ridondante: {audio.shape} " )
    print(f"\n")

    # Crea l'encoder con STFT
    encoder, _ = make_enc_dec(fb_name='stft', kernel_size=512, n_filters=512, sample_rate=16000, padding=256)

    # Passa l'audio tramite l'encoder
    encoded_audio = encoder(audio)

    print(f"forma audio encoded con parte ridondante: {audio.shape} " )
    print(f"\n")

    #prendo solo la parte non ridondante
    #parte_non_ridondante = int((512 / 2) + 1)
    #encoded_audio = encoded_audio[: parte_non_ridondante , :]
    
    #re, im = transforms.reim(encoded_audio)

    print(f"forma audio encoded SENZA parte ridondante:  " )
    print(f"\n")

    # Calcola magnitude e phase
    magn = transforms.mag(encoded_audio, 0)
    #magn = torch.abs(encoded_audio)
    magn = torch.log(magn**2 + 1e-7)
    #magn = magn.unsqueeze(0).unsqueeze(0)
    #magn = magn.permute(0, 1, 3, 2)
   # magn = magn[:,:,:,:-1]


    print(f"dimensioni magnitude PRIMA del resizing: {magn.shape}")
    print(f"\n")

    #angles = torch.angle(encoded_audio)
    #angles = transforms.angle(encoded_audio, 0)
    #angles = angles.unsqueeze(0).unsqueeze(0)
    #angles = angles.permute(0, 1, 3, 2)
    #angles_cos = torch.cos(angles)
    #angles_sin = torch.sin(angles)
    # torch Librosa [1,1,626, 257] (batch, canale, tempo, frequenze)

    #print(f"dimensioni angles PRIMA del resizing: {angles.shape}")
    print(f"\n")

    #angles_cos = angles_cos[:,:,:,:-1]
    #angles_sin = angles_sin[:,:,:,:-1]

    print(f"dimensioni magnitude DOPO del resizing: {magn.shape}")
    print(f"\n")

    #print(f"dimensioni angles DOPO del resizing: {angles_cos.shape}, {angles_sin.shape}")
    print(f"\n")

process_audio_with_stft_encoder(audio_path)



forma audio encoded con parte ridondante: torch.Size([160000]) 


forma audio encoded SENZA parte ridondante:  


dimensioni magnitude PRIMA del resizing: torch.Size([257, 626])




dimensioni magnitude DOPO del resizing: torch.Size([257, 626])






In [87]:
def process_audio_with_free_encoder(audio_path):
    # Carica l'audio
    audio, _ = librosa.load(audio_path, sr=16000, mono = True, res_type = "kaiser_fast")
    audio = audio.squeeze()
    audio = torch.tensor(audio).float()

    #print(f"forma audio encoded con parte ridondante: {audio.shape} " )
    print(f"\n")

    # Crea l'encoder con STFT
    encoder, _ = make_enc_dec(fb_name='free', kernel_size=512, n_filters=512, sample_rate=16000, padding=256)

    # Passa l'audio tramite l'encoder
    encoded_audio = encoder(audio)

    print(f"forma audio encoded con parte ridondante: {encoded_audio.shape} " )
    print(f"\n")

    magn = transforms.mag(encoded_audio)
    #magn = torch.abs(encoded_audio)
    magn = torch.log(magn**2 + 1e-7)
    magn = magn.unsqueeze(0)

    print(f"forma audio encoded con parte ridondante: {magn.shape} " )

 #   magn = magn.unsqueeze(0).unsqueeze(0)
  #  magn = magn.permute(0, 1, 3, 2)
   # magn = magn[:,:,:,:-1]

    angles = transforms.angle(encoded_audio)
    #angles = angles.unsqueeze(0)
    #angles = angles.permute(1, 0, 3, 2)
    angles_cos = torch.cos(angles)
    angles_sin = torch.sin(angles)

    print(f"forma audio encoded con parte ridondante: {angles_cos.shape}, {angles_sin.shape} " )


    


process_audio_with_free_encoder(audio_path)






forma audio encoded con parte ridondante: torch.Size([512, 626]) 


forma audio encoded con parte ridondante: torch.Size([1, 256, 626]) 
forma audio encoded con parte ridondante: torch.Size([256, 626]), torch.Size([256, 626]) 


In [106]:
def process_audio_with_analyticFree_encoder(audio_path):
    # Carica l'audio
    audio, _ = librosa.load(audio_path, sr=16000, mono = True, res_type = "kaiser_fast")
    audio = audio.squeeze()
    audio = torch.tensor(audio).float()

    #print(f"forma audio encoded con parte ridondante: {audio.shape} " )
    print(f"\n")

    # Crea l'encoder con STFT
    encoder, _ = make_enc_dec(fb_name='analytic_free', kernel_size=512, n_filters=512, sample_rate=16000, padding=256)

    # Passa l'audio tramite l'encoder
    encoded_audio = encoder(audio)

    print(f"forma audio encoded con parte ridondante: {encoded_audio} " )
    print(f"\n")

    magn = transforms.mag(encoded_audio)
    #magn = torch.abs(encoded_audio)
    magn = torch.log(magn**2 + 1e-7)
    magn = magn.unsqueeze(0).unsqueeze(0)


 #   magn = magn.unsqueeze(0).unsqueeze(0)
    magn = magn.permute(0, 1, 3, 2)
   # magn = magn[:,:,:,:-1]
    print(f"forma audio encoded con parte ridondante: {magn.shape} " )


    angles = transforms.angle(encoded_audio)
    angles = angles.unsqueeze(0)
   # angles = angles.permute(1, 0, 3, 2)
    angles_cos = torch.cos(angles)
    angles_sin = torch.sin(angles)

    print(f"forma audio encoded con parte ridondante: {angles_cos.shape}, {angles_sin.shape} " )


    


process_audio_with_analyticFree_encoder(audio_path)






forma audio encoded con parte ridondante: tensor([[ 2.7232e-20, -6.6756e-10,  2.6384e-06,  ..., -7.0399e-05,
         -2.5837e-05, -1.5493e-05],
        [ 1.6812e-19,  2.0791e-10,  2.1774e-06,  ..., -1.8285e-04,
         -9.0393e-07, -1.0783e-05],
        [-2.3751e-20, -4.9313e-10,  1.4726e-06,  ..., -3.9446e-05,
         -1.0620e-04,  3.7418e-05],
        ...,
        [-8.5015e-20,  1.9949e-10,  2.0318e-06,  ..., -6.0552e-05,
          5.5319e-05, -7.3575e-06],
        [-7.3239e-20,  1.5759e-09,  2.6088e-06,  ..., -6.1764e-06,
         -1.2886e-06,  7.7219e-05],
        [-1.4278e-19,  2.5104e-10, -4.6750e-06,  ...,  6.1055e-05,
         -3.1502e-05,  1.0085e-04]], grad_fn=<SqueezeBackward0>) 


forma audio encoded con parte ridondante: torch.Size([1, 1, 626, 256]) 
forma audio encoded con parte ridondante: torch.Size([1, 256, 626]), torch.Size([1, 256, 626]) 


In [28]:
import torch
import torch.nn as nn
def process_audio_with_param_sinc_encoder(audio_path):
    # Carica l'audio
    audio, _ = librosa.load(audio_path, sr=16000, mono = True, res_type = "kaiser_fast")
    audio = audio.squeeze()
    audio = torch.tensor(audio).float().unsqueeze(0).unsqueeze(0)

    #print(f"forma audio encoded con parte ridondante: {audio.shape} " )
    print(f"\n")
    print(f"magn shape: {audio.shape} " )


    # Crea l'encoder con STFT
    encoder = nn.Sequential(
        nn.Conv1d(in_channels=1, out_channels=64, kernel_size=7, stride=1, padding=3),
        nn.MaxPool1d(kernel_size=4, stride=4),
        nn.Conv1d(in_channels=64, out_channels=128, kernel_size=5, stride=1, padding=2),
        nn.MaxPool1d(kernel_size=4, stride=4),
        nn.Conv1d(in_channels=128, out_channels=256, kernel_size=3, stride=1, padding=1),
        nn.MaxPool1d(kernel_size=4, stride=4),
        nn.Conv1d(in_channels=256, out_channels=256, kernel_size=3, stride=1, padding=1),
        nn.MaxPool1d(kernel_size=4, stride=4, padding=2)
    )

    # Passa l'audio tramite l'encoder
    encoded_audio = encoder(audio)
    print(f"magn shape: {encoded_audio.shape} " )


    #print(f"forma audio encoded con parte ridondante: {encoded_audio} " )
    print(f"\n")

    magn = transforms.mag(encoded_audio)
    #magn = torch.abs(encoded_audio)
    magn = torch.log(magn**2 + 1e-7)
    magn = magn.unsqueeze(0).unsqueeze(0)


    magn = magn.permute(0, 1, 3, 2)
    print(f"magn shape: {magn.shape} " )


    angles = transforms.angle(encoded_audio)
    angles = angles.unsqueeze(0).unsqueeze(0)
    angles = angles.permute(1, 0, 3, 2)
    angles_cos = torch.cos(angles)
    angles_sin = torch.sin(angles)

    print(f"forma audio encoded con parte ridondante: {angles_cos.shape}, {angles_sin.shape} " )


    


process_audio_with_param_sinc_encoder(audio_path)






magn shape: torch.Size([1, 1, 160000]) 
magn shape: torch.Size([1, 256, 626]) 




RuntimeError: permute(sparse_coo): number of dimensions in the tensor input does not match the length of the desired ordering of dimensions i.e. input.dim() = 5 is not equal to len(dims) = 4